In [1]:
from typing import List
import gradio as gr
import numpy as np

def upload_file(files):
    file_paths = [file.name for file in files]
    return file_paths

def check_if_folder_has_images(file_paths):
    return len(file_paths) > 0

def show_state_of_files(file_paths):
    print(file_paths.value)
    
def rank(images, prompt):
    # randomize order of images
    images = [file.name for file in images]
    return np.random.permutation(images)
    

In [2]:
from distutils.command.upload import upload
import gradio as gr

def build_demo(collection_num_images: int = 0, challenge_num_images: int = 5):
    with gr.Blocks() as demo:
        with gr.Row():
            with gr.Column(scale=1, min_width=224):
                prompt = gr.Textbox(label="prompt-tbox", value="Your prompt here")
        with gr.Row():
            with gr.Column(scale=1, min_width=224):
                rank_status = gr.Button(value="rank", label="ran-btn")
        with gr.Row():
            if collection_num_images > 0:
                collection_images = []
                with gr.Column(scale=collection_num_images, min_width=224):
                    for i in range(collection_num_images):
                        collection_images.append(gr.Image(shape=(224, 224), label=f"collection-image-{i}"))
            with gr.Column(scale=challenge_num_images, min_width=224):
                challenge_images = []
                for i in range(challenge_num_images):
                    challenge_images.append(gr.Image(shape=(224, 224), label=f"challenge-image-{i}"))
            with gr.Column(scale=challenge_num_images, min_width=224):
                ranked_images = []
                for i in range(challenge_num_images):
                    ranked_images.append(gr.Image(shape=(224, 224), label=f"ranked-image-{i}"))
        with gr.Row():
            with gr.Column(scale=2, min_width=224):
                file_output = gr.File(label="challenge-images-folder")
                upload_button = gr.UploadButton("Browse to select a folder with images", file_types=["image"], file_count="multiple")
                upload_button.upload(upload_file, upload_button, file_output)
                upload_button.upload(upload_file, upload_button, challenge_images)
        rank_status.click(fn=rank, inputs=[upload_button, prompt], outputs=ranked_images)
    return demo
demo = build_demo()
demo.launch(share=True, debug=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://2a8bb4d2b2573012.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
